# Process Framework Notebook

## Simple Process

![](simple.png)

### Step 1: Importing the Required Libraries

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.21.1"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.21.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.Abstractions, 1.21.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.21.1-alpha"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using System.ComponentModel;
using System.Reflection;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.DependencyInjection;
/////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////
//Create the kernel
var builder = Kernel.CreateBuilder();
/////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
/////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////
// Build the kernel
var kernel = builder.Build();
/////////////////////////////////////////////////////////////////////////
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0080

/////////////////////////////////////////////////////////////////////////
// Create a new Process
ProcessBuilder process = new("BasicChatBot");
/////////////////////////////////////////////////////////////////////////


Installed Packages Microsoft.SemanticKernel, 1.21.1 Microsoft.SemanticKernel.Process.Abstractions, 1.21.1-alpha Microsoft.SemanticKernel.Process.Core, 1.21.1-alpha Microsoft.SemanticKernel.Process.LocalRuntime, 1.21.1-alpha

## Setup the Classes for the Account Process

### Step 2: Define the Account Process Step Classes


In [2]:
#pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080
/////////////////////////////////////////////////////////////////////////
//Setup all of the classes to be used in the process
////////////////////////////////////////////////////////////////////////


/////////////////////////////////////////////////////////////////////////
//Setup CommonEvents that can be used across multiple processes
public static class CommonEvents
{
    public static readonly string UserInputReceived = nameof(UserInputReceived);
    public static readonly string AssistantResponseGenerated = nameof(AssistantResponseGenerated);
}
/////////////////////////////////////////////////////////////////////////


/////////////////////////////////////////////////////////////////////////
//Setup ScriptedUserInput Step to act as a user
public class ScriptedUserInputStep : KernelProcessStep<UserInputState>
{
    public static class Functions
    {
        public const string GetUserInput = nameof(GetUserInput);
    }

    /// <summary>
    /// The state object for the user input step. This object holds the user inputs and the current input index.
    /// </summary>
    protected UserInputState? _state;

    /// <summary>
    /// Method to be overridden by the user to populate with custom user messages
    /// </summary>
    public virtual void PopulateUserInputs()
    {
        return;
    }

    /// <summary>
    /// Activates the user input step by initializing the state object. This method is called when the process is started
    /// and before any of the KernelFunctions are invoked.
    /// </summary>
    /// <param name="state">The state object for the step.</param>
    /// <returns>A <see cref="ValueTask"/></returns>
    public override ValueTask ActivateAsync(KernelProcessStepState<UserInputState> state)
    {
        state.State ??= new();
        _state = state.State;

        PopulateUserInputs();

        return ValueTask.CompletedTask;
    }

    /// <summary>
    /// Gets the user input.
    /// </summary>
    /// <param name="context">An instance of <see cref="KernelProcessStepContext"/> which can be
    /// used to emit events from within a KernelFunction.</param>
    /// <returns>A <see cref="ValueTask"/></returns>
    [KernelFunction(Functions.GetUserInput)]
    public async ValueTask GetUserInputAsync(KernelProcessStepContext context)
    {
        var userMessage = _state!.UserInputs[_state.CurrentInputIndex];
        _state.CurrentInputIndex++;

        Console.ForegroundColor = ConsoleColor.Yellow;
        Console.WriteLine($"USER: {userMessage}");
        Console.ResetColor();

        // Emit the user input
        await context.EmitEventAsync(new() { Id = CommonEvents.UserInputReceived, Data = userMessage });
    }
}
/////////////////////////////////////////////////////////////////////////

/// <summary>
/// The state object for the <see cref="ScriptedUserInputStep"/>
/// </summary>


/////////////////////////////////////////////////////////////////////////
//Setup UserInput State to hold the user inputs
public record UserInputState
{
    public List<string> UserInputs { get; init; } = [];

    public int CurrentInputIndex { get; set; } = 0;
}
/////////////////////////////////////////////////////////////////////////


/////////////////////////////////////////////////////////////////////////
//Setup DisplayAssistantMessage Step to act as an assistant
public class DisplayAssistantMessageStep : KernelProcessStep
{
    public static class Functions
    {
        public const string DisplayAssistantMessage = nameof(DisplayAssistantMessage);
    }

    [KernelFunction(Functions.DisplayAssistantMessage)]
    public async ValueTask DisplayAssistantMessageAsync(KernelProcessStepContext context, string assistantMessage)
    {
        Console.ForegroundColor = ConsoleColor.Blue;
        Console.WriteLine($"ASSISTANT: {assistantMessage}\n");
        Console.ResetColor();

        // Emit the assistantMessageGenerated
        await context.EmitEventAsync(new() { Id = CommonEvents.AssistantResponseGenerated, Data = assistantMessage });
    }
}
/////////////////////////////////////////////////////////////////////////


/////////////////////////////////////////////////////////////////////////
//Setup Intro Step to get the first request from the user
 private sealed class IntroStep : KernelProcessStep
    {
        /// <summary>
        /// Prints an introduction message to the console.
        /// </summary>
        [KernelFunction]
        public void PrintIntroMessage()
        {
            System.Console.WriteLine("Welcome to Processes in Semantic Kernel.\n");
        }
    }
/////////////////////////////////////////////////////////////////////////


    /// <summary>
    /// A step that elicits user input.
    /// </summary>

    /////////////////////////////////////////////////////////////////////////
    //Setup ChatUserInput Step to get the chat input
    private sealed class ChatUserInputStep : ScriptedUserInputStep
    {
        public override void PopulateUserInputs()
        {
            if (_state != null)
            {
                _state.UserInputs.Add("Hello");
                _state.UserInputs.Add("How tall is the tallest mountain?");
                _state.UserInputs.Add("How low is the lowest valley?");
                _state.UserInputs.Add("How wide is the widest river?");
                _state.UserInputs.Add("exit");
            }
        }
    }
    /////////////////////////////////////////////////////////////////////////


    /// <summary>
    /// A step that takes the user input from a previous step and generates a response from the chat completion service.
    /// </summary>


    /////////////////////////////////////////////////////////////////////////
    //Setup ChatBotResponse Step to get the response
    private sealed class ChatBotResponseStep : KernelProcessStep<ChatBotState>
    {
        public static class Functions
        {
            public const string GetChatResponse = nameof(GetChatResponse);
        }

        /// <summary>
        /// The internal state object for the chat bot response step.
        /// </summary>
        internal ChatBotState? _state;

        /// <summary>
        /// ActivateAsync is the place to initialize the state object for the step.
        /// </summary>
        /// <param name="state">An instance of <see cref="ChatBotState"/></param>
        /// <returns>A <see cref="ValueTask"/></returns>
        public override ValueTask ActivateAsync(KernelProcessStepState<ChatBotState> state)
        {
            _state = state.State ?? new();
            _state.ChatMessages ??= new();
            return ValueTask.CompletedTask;
        }

        /// <summary>
        /// Generates a response from the chat completion service.
        /// </summary>
        /// <param name="context">The context for the current step and process. <see cref="KernelProcessStepContext"/></param>
        /// <param name="userMessage">The user message from a previous step.</param>
        /// <param name="_kernel">A <see cref="Kernel"/> instance.</param>
        /// <returns></returns>
        [KernelFunction(Functions.GetChatResponse)]
        public async Task GetChatResponseAsync(KernelProcessStepContext context, string userMessage, Kernel _kernel)
        {
            _state!.ChatMessages.Add(new(AuthorRole.User, userMessage));
            IChatCompletionService chatService = _kernel.Services.GetRequiredService<IChatCompletionService>();
            ChatMessageContent response = await chatService.GetChatMessageContentAsync(_state.ChatMessages).ConfigureAwait(false);
            if (response == null)
            {
                throw new InvalidOperationException("Failed to get a response from the chat completion service.");
            }

            System.Console.ForegroundColor = ConsoleColor.Yellow;
            System.Console.Write("Assistant: ");
            System.Console.ResetColor();
            System.Console.WriteLine(response.Content);

            // Update state with the response
            _state.ChatMessages.Add(response);

            // emit event: assistantResponse
            await context.EmitEventAsync(new KernelProcessEvent { Id = ChatBotEvents.AssistantResponseGenerated, Data = response });
        }
    }
    /////////////////////////////////////////////////////////////////////////



    /// <summary>
    /// The state object for the <see cref="ChatBotResponseStep"/>.
    /// </summary>

    
    private sealed class ChatBotState
    {
        internal ChatHistory ChatMessages { get; set; } = new();
    }

    /// <summary>
    /// A class that defines the events that can be emitted by the chat bot process. This is
    /// not required but used to ensure that the event names are consistent.
    /// </summary>
    private static class ChatBotEvents
    {
        public const string StartProcess = "startProcess";
        public const string IntroComplete = "introComplete";
        public const string AssistantResponseGenerated = "assistantResponseGenerated";
        public const string Exit = "exit";
    }

### Step 3: Create the Step Objects

In [3]:
 #pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080
 
        var introStep = process.AddStepFromType<IntroStep>();
        var userInputStep = process.AddStepFromType<ChatUserInputStep>();
        var responseStep = process.AddStepFromType<ChatBotResponseStep>();

### Step 4: Define the patterns for each step

In [4]:
 #pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080

        process
            .OnExternalEvent(ChatBotEvents.StartProcess)
            .SendEventTo(new ProcessFunctionTargetBuilder(introStep));

        // When the intro is complete, notify the userInput step
        introStep
            .OnFunctionResult(nameof(IntroStep.PrintIntroMessage))
            .SendEventTo(new ProcessFunctionTargetBuilder(userInputStep));

        // When the userInput step emits an exit event, send it to the end steprt
        userInputStep
            .OnFunctionResult("GetUserInput")
            .StopProcess();

        // When the userInput step emits a user input event, send it to the assistantResponse step
        userInputStep
            .OnEvent(CommonEvents.UserInputReceived)
            .SendEventTo(new ProcessFunctionTargetBuilder(responseStep, parameterName: "userMessage"));

        // When the assistantResponse step emits a response, send it to the userInput step
        responseStep
            .OnEvent(ChatBotEvents.AssistantResponseGenerated)
            .SendEventTo(new ProcessFunctionTargetBuilder(userInputStep));

        KernelProcess kernelProcess = process.Build();

### Start the Process

In [5]:
 #pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080
 
// Start the process with an initial external event
var runningProcess = await kernelProcess.StartAsync(kernel, new KernelProcessEvent() { Id = ChatBotEvents.StartProcess, Data = null });

Welcome to Processes in Semantic Kernel.

USER: Hello
Assistant: Hi there! How can I assist you today?
USER: How tall is the tallest mountain?
Assistant: The tallest mountain in the world is Mount Everest, which stands at approximately 29,032 feet (8,848 meters) above sea level. This measurement can vary slightly due to snow and other atmospheric conditions. Mount Everest is located in the Himalaya mountain range, on the border between Nepal and the Tibet Autonomous Region of China.
USER: How low is the lowest valley?
Assistant: The lowest point on Earth's land surface is the shoreline of the Dead Sea Depression. The Dead Sea itself sits at approximately 1,410 feet (430 meters) below sea level, but this depth can vary due to changes in water level. The Dead Sea Depression is located in the Jordan Rift Valley, bordered by Jordan to the east and Israel and the West Bank to the west. This unique area is well-known for its extremely high salinity levels, which create a harsh environment fo